In [35]:
# !pip install matplotlib

In [37]:
import collections
import itertools
import numpy as np

import util
from spam import get_words, create_dictionary, transform_text, fit_naive_bayes_model
 

list(itertools.chain(*[get_words(message) for message in ['Привет как дела', 'Привет как дела']]))
# collections.Counter(
#     itertools.chain(get_words(message) for message in ['Привет'])
#     )

['привет', 'как', 'дела', 'привет', 'как', 'дела']

In [32]:
matrix = np.array([
   [0, 1, 0, 1],
   [0, 1, 1, 1],
   [1, 1, 1, 1]
])
labels = np.array([
    1, 0, 1
])
V = matrix.shape[1]
# 1 + labels @ matrix
# V + sum(labels @ matrix)
(1 + labels @ matrix) / (V + sum(labels @ matrix))
(1 + (1 - labels) @ matrix) / (V + sum((1 - labels) @ matrix))

array([0.14285714, 0.28571429, 0.28571429, 0.28571429])

In [ ]:
phi_k_y = (matrix & labels).sum(axis=0)

In [42]:
train_messages, train_labels = util.load_spam_dataset('spam_train.tsv')
val_messages, val_labels = util.load_spam_dataset('spam_val.tsv')
test_messages, test_labels = util.load_spam_dataset('spam_test.tsv')

dictionary = create_dictionary(train_messages)

print('Size of dictionary: ', len(dictionary))

util.write_json('spam_dictionary', dictionary)

train_matrix = transform_text(train_messages, dictionary)

np.savetxt('spam_sample_train_matrix', train_matrix[:100,:])

val_matrix = transform_text(val_messages, dictionary)
test_matrix = transform_text(test_messages, dictionary)

naive_bayes_model = fit_naive_bayes_model(train_matrix, train_labels)

Size of dictionary:  1757


In [85]:
# test_labels

In [44]:
matrix = test_matrix

In [50]:
phi_k_1

array([7.77302759e-05, 6.14069180e-03, 7.77302759e-05, ...,
       4.66381656e-04, 3.88651380e-04, 7.77302759e-05])

In [66]:
(np.exp(np.log(matrix @ phi_k_1)) * phi_1)

C:\Users\azh.makanov\AppData\Local\Temp\ipykernel_14276\2884469944.py:1: RuntimeWarning: divide by zero encountered in log
  (np.exp(np.log(matrix @ phi_k_1)) * phi_1)


array([1.37034448e-02, 4.36890541e-04, 1.27870402e-04, 1.60690472e-02,
       2.21642031e-03, 3.74020927e-03, 5.26399823e-03, 1.01337294e-02,
       6.35089665e-03, 7.03287213e-04, 3.86807967e-03, 1.26165464e-02,
       3.67627407e-03, 6.27630558e-03, 9.84602098e-03, 1.33198336e-02,
       1.38526269e-04, 7.77878281e-03, 1.39591856e-03, 2.13117337e-05,
       1.49821488e-02, 1.00165148e-02, 5.67957704e-03, 7.98124428e-03,
       1.14550569e-02, 4.64595795e-03, 1.56641243e-03, 1.69428283e-03,
       4.53939928e-03, 4.03857354e-03, 5.22137476e-04, 1.39378739e-02,
       6.81975479e-04, 2.15248511e-03, 4.74186075e-03, 2.05551672e-02,
       2.09920577e-03, 3.83611207e-04, 4.70989315e-03, 7.40582747e-03,
       3.04757792e-02, 1.52378896e-03, 1.92338397e-02, 7.52304200e-03,
       9.07879857e-03, 3.72955340e-04, 5.81810331e-03, 2.08428756e-02,
       1.38313152e-02, 1.94469570e-02, 5.51973903e-03, 5.93531784e-03,
       2.54675218e-03, 3.05823379e-03, 2.22174824e-02, 3.43118913e-03,
      

In [90]:
phi_1, phi_k_1, phi_k_0 = naive_bayes_model
eps = 1e-8
m_1 = np.exp(matrix @ np.log(eps + phi_k_1)) * phi_1
m_0 = np.exp(matrix @ np.log(eps + phi_k_0)) * (1 - phi_1)

probabilties = m_1 / (m_1 + m_0)
predictions = (probabilties > 0.5).astype(np.int32)

In [93]:
metric = np.log(phi_k_1 / phi_k_0)

In [99]:
idx2word = {idx: word for word, idx in dictionary.items()}
[idx2word[idx] for idx in np.argsort(-metric)[:5]]

['claim', 'won', 'prize', 'tone', 'urgent!']